In [1]:
from py2neo import authenticate, Graph, Relationship
graph = Graph("http://localhost:7474/db/data/")
from py2neo import Node, Relationship

In [2]:
def cypher_statement_start(location, name, date, day):
    location = location.lower()
    if not location or not name:
        raise ValueError("Must have a location and a name!")
        
    statement = """MATCH (airport1: Airport {{ {location} : '{name}' }} ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
    """.format(location=location, name=name)
    
    where = None
    
    if type(date) == list and type(date[0]) == int :
        data_type = type(date[1])
        if  data_type == int:
            where = 'WHERE ad1.day >= {date1} AND ad1.day <= {date2}'.format(date1=min(date), date2=max(date))
        elif date[1] is None:
            where = 'WHERE ad1.day = {date}'.format(date=date[0])
        elif date[1] in ['<=','>=','=']:
            where = 'WHERE ad1.day {sign} {date}'.format(date=date[0], sign=date[1])
        else:
            raise ValueError('Invalid sign type ' + str(date[1])) 
    else:
        if not date is None:
            raise ValueError('Invalid date type ' + str(date))
        
    if where:
        return statement  + where
    
    return statement 
    
    
def test_start(tup):
    print "TESTING:", tup
    statement = cypher_statement_start(*tup)
    statement = statement + " RETURN ad1"
    print statement, "\nLENGTH:", len(list(graph.run(statement)))
    print 

In [3]:
start_test_1 = ('City', 'Paris', None, None)
start_test_2 = ('City', 'London', [7, None], None)
start_test_3 = ('City', 'London', [7, 8], None)
start_test_4 = ('City', 'London', [7, '<='], None)
start_test_5 = ('City', 'London', [7, '>='], None)

#test_start(start_test_2)
test_start(start_test_5)
test_start(start_test_4)
test_start(start_test_3)
test_start(start_test_2)
test_start(start_test_1)


TESTING: ('City', 'London', [7, '>='], None)
MATCH (airport1: Airport { city : 'London' } ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
    WHERE ad1.day >= 7 RETURN ad1 
LENGTH: 36

TESTING: ('City', 'London', [7, '<='], None)
MATCH (airport1: Airport { city : 'London' } ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
    WHERE ad1.day <= 7 RETURN ad1 
LENGTH: 28

TESTING: ('City', 'London', [7, 8], None)
MATCH (airport1: Airport { city : 'London' } ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
    WHERE ad1.day >= 7 AND ad1.day <= 8 RETURN ad1 
LENGTH: 8

TESTING: ('City', 'London', [7, None], None)
MATCH (airport1: Airport { city : 'London' } ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
    WHERE ad1.day = 7 RETURN ad1 
LENGTH: 4

TESTING: ('City', 'Paris', None, None)
MATCH (airport1: Airport { city : 'Paris' } ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
     RETURN ad1 
LENGTH: 30



In [4]:
def cypher_statement_stop(location, name, stop_number):
    no = str(stop_number*2)
    airport_day = "(ad" + no + ":AirportDay)"
    airport = "(:Airport {" +location.lower() +": '"+ name +"' })"
    statement = airport_day + "  - [:HAS_AIRPORT] -> " + airport  
    return statement
    
cypher_statement_stop('City', 'Paris', 4)
    

"(ad8:AirportDay)  - [:HAS_AIRPORT] -> (:Airport {city: 'Paris' })"

In [5]:
def create_statement(start, stops, destination, inOrder = True, return_flight = True):
    
    processed_statements = []
    
    processed_statements.append(cypher_statement_start(*start)) 
    
    
    for index, stop in enumerate(stops, start = 1):
        
        if index > 2:
            print "\n WE NEED A NEW MARKET \n"
            
        
        processed_statements.append( cypher_statement_stop(stop[0], stop[1], index) )
    
    if destination:
        processed_statements.append( cypher_statement_stop(destination[0], destination[1], len(processed_statements) + 1) )
        
    for ps in processed_statements:
        print ps

start = ('City','Dublin',[7,None], None)
stop1 = ('City','Oslo',4,'Airport')
stop2 = ('City','Paris',3,'City')
destination = ('City','Cork',3, None)
stops = [stop1,stop2]

create_statement(start,stops, destination)

MATCH (airport1: Airport { city : 'Dublin' } ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
    WHERE ad1.day = 7
(ad2:AirportDay)  - [:HAS_AIRPORT] -> (:Airport {city: 'Oslo' })
(ad4:AirportDay)  - [:HAS_AIRPORT] -> (:Airport {city: 'Paris' })
(ad8:AirportDay)  - [:HAS_AIRPORT] -> (:Airport {city: 'Cork' })


In [6]:
def create_statement(start, stops, destination, inOrder = True, return_flight = True):
    
    processed_statements = []
    
    processed_statements.append(cypher_statement_start(*start))
    all_tuples = [start]
    
    
    for index, stop in enumerate(stops, start = 1):
                  
        s = cypher_statement_stop(stop[0], stop[1], index)
        print "NEW ST", s
                                  
        processed_statements.append( s) 
        all_tuples.append(stop)
    
    if destination:
        processed_statements.append( cypher_statement_stop(destination[0], destination[1], len(processed_statements)  )) 
        
    pairs = ((x*2-1,x*2, statement) for x, statement in enumerate(processed_statements))
    
    statement = ""
    
    for index, pair in enumerate(pairs):
        
        early, late, ps = pair
        
        print index, early, late, ps
        
        if index >= 2: ## FIND NEW MARKET
            prev = early - 1
            duration, location = all_tuples[index-1][2:]
            
            print "WE NEED A NEW MArKET"
            print duration, location
            
            if not location or location == 'Airport':
                market1 = "(ad" + str(early-1) + ") "
                market2 = "(ad" + str(early) + ": AirportDay)"
                add = market1 + " - [:NEW_MARKET {days: " + str(duration) + "}] - " + market2
                print "ADD MARKET", add
                statement = statement + "\nMATCH " + add
            
        if index >= 1: ## Connect Flight
            print "WE NEED TO ADD FLIGHT", processed_statements[index]
            statement = statement + "\nMATCH (ad" + str(early) 
            statement = statement + ") - [f" + str(index) + ":FLIGHT] -> " + processed_statements[index]
            
        else:
            statement = statement + processed_statements[index]
        
                
            
        print "\n===========\n", statement, "\n===========\n"
    for ps in processed_statements:
        all_tuples.append(destination)
        print ps

start = ('City','Dublin',[7,None], None)
stop1 = ('City','Oslo',1,'Airport')
stop2 = ('City','Paris',2,None)
destination = ('City','Cork',3, None)
stops = [stop1,stop2]

create_statement(start,stops, destination)

NEW ST (ad2:AirportDay)  - [:HAS_AIRPORT] -> (:Airport {city: 'Oslo' })
NEW ST (ad4:AirportDay)  - [:HAS_AIRPORT] -> (:Airport {city: 'Paris' })
0 -1 0 MATCH (airport1: Airport { city : 'Dublin' } ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
    WHERE ad1.day = 7

MATCH (airport1: Airport { city : 'Dublin' } ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
    WHERE ad1.day = 7 

1 1 2 (ad2:AirportDay)  - [:HAS_AIRPORT] -> (:Airport {city: 'Oslo' })
WE NEED TO ADD FLIGHT (ad2:AirportDay)  - [:HAS_AIRPORT] -> (:Airport {city: 'Oslo' })

MATCH (airport1: Airport { city : 'Dublin' } ) <- [:HAS_AIRPORT] - (ad1:AirportDay)
    WHERE ad1.day = 7
MATCH (ad1) - [f1:FLIGHT] -> (ad2:AirportDay)  - [:HAS_AIRPORT] -> (:Airport {city: 'Oslo' }) 

2 3 4 (ad4:AirportDay)  - [:HAS_AIRPORT] -> (:Airport {city: 'Paris' })
WE NEED A NEW MArKET
1 Airport
ADD MARKET (ad2)  - [:NEW_MARKET {days: 1}] - (ad3: AirportDay)
WE NEED TO ADD FLIGHT (ad4:AirportDay)  - [:HAS_AIRPORT] -> (:Airport {city: 'Paris' })

MATCH (airport1: